In [13]:
client.models.list().page 

[Model(name='models/embedding-gecko-001', display_name='Embedding Gecko', description='Obtain a distributed representation of a text.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1024, output_token_limit=1, supported_actions=['embedText', 'countTextTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gemini-1.0-pro-vision-latest', display_name='Gemini 1.0 Pro Vision', description='The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=12288, output_token_limit=4096, supported_actions=['generateContent', 'countTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gem

In [89]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate textual descriptions pairs of visual discriminative features to identify whether the central region of an histopathological image patch contains tumor tissue or not. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section.
    Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.

Current Top 10 prompt pairs:
('Scant background inflammation.', 'Prominent lymphoplasmacytic infiltrate surrounding tumor nests.'), Score: 60
('Normal lymphocyte morphology.', 'Presence of Reed-Sternberg cells or Hodgkin-like cells.'), Score: 61
('Normal mitotic activity.', 'Increased mitotic rate with atypical mitotic figures.'), Score: 63
('Lymphocytes arranged in organized follicles.', 'Tumor cells growing in sheets with loss of follicular architecture.'), Score: 69
('No necrosis observed.', 'Areas of tumor necrosis with karyorrhexis.'), Score: 70
('Uniformly sized and shaped lymphocytes.', 'Pleomorphic lymphocytes with irregular nuclei.'), Score: 71
('No emperipolesis.', 'Emperipolesis - intact lymphocytes within tumor cells.'), Score: 75
('Scant cytoplasm in lymphocytes.', 'Abundant and eosinophilic cytoplasm in tumor cells.'), Score: 76
('No evidence of necrosis.', 'Areas of tumor necrosis with karyorrhexis.'), Score: 79
('No prominent nucleoli.', 'Large, prominent nucleoli within tumor cell nuclei.'), Score: 90

Last Instruction: Write 10 new prompt pairs that are different from the old ones and has a score as high as possible.
The last instruction did not improve the score significantly, so we need to change our approach.

This instruction will be used in the following setting:
"<Problem Description and task to be performed>
<Current Top 10 Prompts: ....>
<Instruction>"

Exemplar Instructions:
Write 10 new prompt pairs that are different from the old ones and has a score as high as possible.
Write 10 new prompt pairs that are more similar to the high scoring prompts
Write 10 new prompt pairs by paraphrasing each of the above. Each pair should have distinct language style.
Write 10 new prompt pairs appending rare or borderline patterns which are easily misclassified to score as high as possible.
Write 10 new prompt pairs by combining multiple medical concepts.
Write 10 new prompt pairs expanding each prompt by appending expert biomedical knowledge.
Write 10 new prompt pairs that adds quantitative cues to the qualitative prompts given above.

Get creative and write a new generalized instruction to improve the score.
Output in a <instruction> tag. Let's think step-by-step,
""")
    print(response.text)

Okay, let's analyze the situation. The previous instruction ("Write 10 new prompt pairs that are different from the old ones and has a score as high as possible.") didn't yield significant improvement. The high-scoring prompts consistently focus on *specific morphological changes* indicative of malignancy – things like nuclear features, mitotic activity, architectural disruption, and presence of atypical cells. Simply asking for "different" prompts doesn't guide the generation towards these crucial features.

The goal is to generate prompts that highlight *discriminative* features. We need to focus on characteristics that strongly suggest tumor presence *versus* normal lymph node tissue. The best prompts aren't just "tumor cells are different," but *how* they are different.

Considering the exemplar instructions, the "combining multiple medical concepts" and "expanding each prompt by appending expert biomedical knowledge" approaches seem promising. However, let's refine this. We need t

In [2]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate distinct textual descriptions pairs of visual discriminative features to identify whether the central region of an histopathological image patch contains tumor tissue or not. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section.
    Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.

Current Top 10 prompt pairs:
1. ('Stroma is collagenous and sparsely cellular.', 'Stroma is desmoplastic and densely cellular with tumor infiltration.'), Score: 60
2. ('Mitotic activity is rare and normal.', 'Mitotic activity is frequent and abnormal (atypical mitoses).'), Score: 60
3. ('Sinuses show normal flow.', 'Sinuses are obstructed by tumor cells.'), Score: 64
4. ('No evidence of necrosis or karyorrhexis.', 'Presence of tumor cell necrosis and karyorrhexis.'), Score: 65
5. ('No evidence of extracellular mucin.', 'Abundant extracellular mucin production by tumor cells.'), Score: 72
6. ('No emperipolesis.', 'Tumor cells exhibiting emperipolesis (intracellular invasion of lymphocytes).'), Score: 78
7. ('Cells exhibit uniform morphology.', 'Cells exhibit marked pleomorphism and variation in size and shape.'), Score: 83
8. ('No increased eosinophilic cytoplasm.', 'Increased eosinophilic cytoplasm in tumor cells.'), Score: 85
9. ('No increase in mitotic activity.', 'Increased mitotic activity and frequent mitoses are observed.'), Score: 86
10. ('No evidence of tumor-induced lymphoid depletion.', 'Lymphoid depletion around tumor nests.'), Score: 90

Write 10 new prompt pairs different to above to improve the score, think slowly and formulate a strategy. 
Output as python code in the format - prompts: list[tuple[negative: str, positive: str]]. Let's think step-by-step.
""")
    print(response.text)

Okay, let's analyze the existing prompt pairs and formulate a strategy to generate new, high-scoring ones.

**Strategy:**

The top-performing pairs generally focus on:

*   **Clear contrasts:**  The negative and positive statements are distinctly different and easily observable.
*   **Specific pathological features:** They don't just say "tumor present" but *how* it's present (necrosis, pleomorphism, mucin, etc.).
*   **Quantifiable/Observable changes:** "Frequent" vs. "Rare", "Abundant" vs. "No evidence", "Increased" vs. "No increase".
*   **Features related to tumor-stroma interaction:** Lymphoid depletion, desmoplasia.

We need to identify features that are commonly altered in lymph node metastases and formulate contrasting pairs.  I'll focus on features related to cell morphology, architecture, and the immune response. I'll also try to incorporate features that are relatively easy for a pathologist to recognize.

Here are 10 new prompt pairs, aiming for scores above 80:

```python
